In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics.api import qqplot
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.api import tsa
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('monthly-sunspots.csv', index_col = 'YEAR')
df

In [ ]:
df.plot(figsize=(15,8), color = 'purple')

In [ ]:
fig = plt.figure(figsize=(15,8))
ax1 = fig.add_subplot(211)
fig = plot_acf(df.values.squeeze(), lags=40, ax=ax1, color = 'r', title='Original Autocorrelation')
ax2 = fig.add_subplot(212)
fig = plot_pacf(df, lags=40, ax=ax2, color = 'g', title='Original Partial Autocorrelation')

In [ ]:
arma20 = ARIMA(df, order=(2,0,0)).fit()
display(arma20.params)
display(arma20.aic, arma20.bic, arma20.hqic)

In [ ]:
arma30 = ARIMA(df, order=(3,0,0)).fit()
display(arma30.params)
display(arma30.aic, arma30.bic, arma30.hqic)

In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111)
ax = arma30.resid.plot(ax=ax)

In [ ]:
resid = arma30.resid
stats.normaltest(resid)

In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111)
fig = qqplot(resid, line='q', ax=ax, fit=True)

In [ ]:
fig = plt.figure(figsize=(15,8))
ax1 = fig.add_subplot(211)
fig = plot_acf(resid.values.squeeze(), lags=40, ax=ax1, color='g', title='Residual Autocorrelation')
ax2 = fig.add_subplot(212)
fig = plot_pacf(resid, lags=40, ax=ax2, color='r', title='Residual Partial Autocorrelation')

In [ ]:
r, q, p = tsa.acf(resid.values.squeeze(), fft=True, qstat=True)
data = np.c_[np.arange(1, 25), r[1:], q, p]

In [ ]:
df2 = pd.DataFrame(data, columns = ['Lag', 'AC', 'Q', 'Prob(>Q)'])
df2.set_index('Lag')

In [ ]:
predict_sunspots = arma30.predict('1990', '2012', dynamic=True)
predict_sunspots = predict_sunspots['1990-12-31':'2008-12-31']
predict_sunspots = pd.DataFrame(predict_sunspots)
predict_sunspots.head()

In [ ]:
np.mean((df['1990-12-31':'2008-12-31':1]['SUNACTIVITY'].values  - predict_sunspots['predicted_mean'].values))